In [1]:
# ! pip install "numpy<2" --upgrade
! pip uninstall numpy -y
! pip install numpy==1.26.4
## restart session

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

import numpy as np
np.__version__

'1.26.4'

In [7]:
pip uninstall scikit-surprise -y


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-surprise


  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached scipy-1.16.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached scipy-1.16.2-cp312-cp312-win_amd64.whl (38.6 MB)
Failed to build scikit-surprise
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for scikit-surprise (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [155 lines of output]
      C:\Users\NB Milagros Irusta\AppData\Local\Temp\pip-build-env-vml7mmot\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ***************************

In [4]:
import surprise as sp
import pandas as pd
import sqlite3

ModuleNotFoundError: No module named 'surprise'

In [ ]:
conn = sqlite3.connect("../datos/qll.db")

df = pd.read_sql("""
    SELECT *
      FROM interacciones
     WHERE id_libro IN (SELECT id_libro FROM interacciones GROUP BY 1 HAVING count(*) > 20)
       AND id_lector IN (SELECT id_lector FROM interacciones GROUP BY 1 HAVING count(*) > 20)
     """, conn)

print(df.shape)
df.head()

In [ ]:
reader = sp.reader.Reader(rating_scale=(1, 10))

data = sp.dataset.Dataset.load_from_df(df[['id_lector', 'id_libro', 'rating']], reader) # cuidado con el orden
trainset, testset = sp.model_selection.train_test_split(data, test_size=0.2, random_state=42)

# Algoritmos de predicción

## BaselineOnly

In [ ]:
algo = sp.prediction_algorithms.baseline_only.BaselineOnly(verbose=False)

## KNN Basic

In [ ]:
sim_options = {
    'name': 'pearson', #'cosine', 'pearson_baseline'
    'user_based': True, #False,
}

algo = sp.prediction_algorithms.knns.KNNBasic(k=40, min_k=5, sim_options=sim_options, random_state=42)
algo.fit(trainset)
predictions = algo.test(testset)

print(f"RMSE: { sp.accuracy.rmse(predictions) }")

## SVD

In [ ]:
algo = sp.prediction_algorithms.matrix_factorization.SVD(n_factors=500, n_epochs=20, random_state=42)

algo.fit(trainset)
predictions = algo.test(testset)

print(f"RMSE: { sp.accuracy.rmse(predictions) }")

## NMF

In [ ]:
algo = sp.prediction_algorithms.matrix_factorization.NMF(n_factors=500, n_epochs=20, random_state=42)

algo.fit(trainset)
predictions = algo.test(testset)

print(f"RMSE: { sp.accuracy.rmse(predictions) }")

# Predicción

In [ ]:
%%time
conn = sqlite3.connect("qll.db")
df = pd.read_sql('SELECT * FROM interacciones;', conn)

reader = sp.reader.Reader(rating_scale=(1, 10))
data = sp.dataset.Dataset.load_from_df(df[['id_lector', 'id_libro', 'rating']], reader) # cuidado con el orden

In [ ]:
%%time

# algoritmo
algo = sp.prediction_algorithms.baseline_only.BaselineOnly(verbose=False)
algo.fit(data.build_full_trainset())

In [ ]:
sp.dump.dump("algo.pkl", algo=algo)
del algo

In [ ]:
_, algo = sp.dump.load("algo.pkl")

In [ ]:
lector_id = "popocito"

libro_id = "reina-roja"
print(libro_id, algo.predict(lector_id, libro_id, 9))
print(libro_id, algo.predict(lector_id, libro_id, 9).est)

libro_id = "la-novia-rebelde"
print(libro_id, algo.predict(lector_id, libro_id, 7).est)
print(libro_id, algo.predict(lector_id, libro_id, 7).est)